In [16]:
# 导包
from notebookutils import mssparkutils  
from openpyxl import load_workbook
import pandas as pd
import os
from datetime import datetime,timedelta    
from pyspark.sql.types import StructType, StructField, StringType
import pyspark.sql.functions as F

In [17]:
# 方法
def schema_from_pandas(df):
    """读取pd文件schema"""
    schema = StructType([StructField(str(col),StringType(),True) for col in df.columns]) 
    return schema

In [18]:
# todo 1：数据载入
ods_reimburse = 'abfss://data-warehouse-ods@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/ods_mau_travel_app_cn_mf.csv'
df_pd = pd.read_csv(ods_reimburse).fillna('') 

schema = schema_from_pandas(df_pd)
df  = spark.createDataFrame(df_pd,schema=schema)


# todo 2:数据准备
cols = {
    'Project' : 'project'
    ,'Key'  :   'TA_key'
    ,'Summary'  :   'summary'
    ,'Issue Type'   :   'issue_type'
    ,'Status'   :   'status'
    ,'Priority' :   'priority'
    ,'Resolution'   :   'resolution'
    ,'Current Assignee' :   'current_assignee'
    ,'Applicant'    :   'applicant'
    ,'Created'  :   'created_date_time'
    ,'Updated'  :   'updated_date_time'
    ,'Due Date' :   'due_date'
    ,'Description'  :   'description1'
    ,'Advance Necessary?*'  :   'advance_necessary'
    ,'Advance Amount(CNY)'  :   'advance_amount_in_cny'
    ,'Book Air Ticket*' :   'book_air_ticket'
    ,'International Assignment' :   'international_assignment'
    ,'Start Date'   :   'start_date'
    ,'Total Hours'  :   'total_hours'
    ,'Approved Training Application Number' :   'approved_training_application_number'
    ,'End Date' :   'end_date'
    ,'Training Type'    :   'training_type'
    ,'Training Venue'   :   'training_venue'
    ,'due date call result' :   'due_date_call_result'
    ,'Validation Result'    :   'validation_result'
    ,'apcntr.field.trainingProvider'    :   'training_provider'
    ,'apcntr.field.trainingType'    :   'training_type_apcntr'
    ,'apcntr.field.trainingNeedsCaused' :   'training_needs_caused'
    ,'apcntr.field.thisTrainingCost'    :   'training_cost'
    ,'apcntr.field.tmpTrainingType' :   'tmp_training_type'
    ,'To Place Validation Result'   :   'to_place_validation_Result'
    ,'To Place Validation'  :   'to_place_validation'
    ,'apcntr.field.estimatedTravelExpenses' :   'estimated_travel_expenses'
    ,'from:'    :   'from'
    ,'apcntr.field.participantNumber'   :   'participantNumber'
    ,'to:'  :   'to'
    ,'apcntr.field.estimatedOtherCost'  :   'estimated_other_cost'
    ,'apcntr.field.estimatedTotalCost'  :   'estimated_total_cost'
    ,'common.field.employee.companycode'    :   'employee_company_code'
    ,'From Place*'  :   'from_place'
    ,'To Place*'    :   'to_place'
    ,'From Place Validation Result' :   'from_place_validation_result'
    ,'From Place Validation'    :   'from_place_validation'
    ,'From City MD' :   'from_city_MD'
    ,'Org.Unit' :   'org_unit'
    ,'Charge to*'   :   'charge_to'
    ,'Charged to Company*'  :   'charged_to_company'
    ,'Charged to Cost Center*'  :   'charged_to_cost_center'
    ,'Temp Dates Between'   :   'temp_dates_between'
    ,'Internal order number*'   :   'internal_order_number'
    ,'Report To'    :   'report_to'
    ,'Application reason*'  :   'application_reason'
    ,'Tel'  :   'Telephone1'
    ,'Employee Number (WorkON Profile)' :   'employee_number_workon_profile'
    ,'Telephone:'   :   'Telephone2'
    ,'Training Objectives'  :   'training_objectives'
    ,'Training Fee' :   'training_fee'
    ,'Applied Training Topics'  :   'applied_training_topics'
    ,'Currency' :   'currency'
    ,'CCView'   :   'CCView'
    ,'Supervisor'   :   'supervisor'
    ,'Advance Return Confirmation'  :   'advance_return_confirmation'
    ,'Travel back date' :   'travel_back_date'
    ,'Training Days'    :   'training_days'
    ,'Personnel Number*'    :   'personnel_number'
    ,'Approvers*'   :   'approvers'
    ,'Advance Amend Account'    :   'advance_amend_account'
    ,'Global CC'    :   'global_CC'
    ,'Planned Duration*:'   :   'planned_duration'
    ,'Planned Start Date*:' :   'planned_start_date'
    ,'Planned Start Date Text'  :   'planned_start_date_text'
    ,'Planned End Date*:'   :   'planned_end_date'
    ,'Planned End Date Text'    :   'planned_end_date_text'
    ,'Global CC 2'  :   'global_CC_2'
    ,'Travel Arranger*' :   'travel_arranger'
    ,'Travel to *'  :   'travel_to'
    ,'Created date' :   'created_date'
    ,'Name' :   'name'
    ,'Applicant NT/Display Name'    :   'applicant_NT_display_name'
    ,'Apply Visa?'  :   'apply_visa'
    ,'Visa To Where'    :   'visa_to_where'
    ,'Visa arranger'    :   'visa_arranger'
    ,'Applied Training Request Number'  :   'applied_training_request_number'
    ,'Expense Actual Booking Cost Center*'  :   'expense_actual_booking_cost_center'
    ,'Training Related*'    :   'training_related'
    ,'Travel Type*.1' :   'travel_type'
    ,'※Pick up at:' :   'pick_up_at'
    ,'※Pick up at:' :   'pick_up_at'
    ,'to:'  :   'to'
    ,'Visa Application:'    :   'visa_application'
    ,'Special Requests:'    :   'special_requests'
    ,'Name on Passport:*'   :   'name_on_passport'
    ,'Personal ID No.*' :   'personal_ID_No'
    ,'Passport Number:*'    :   'passport_number'
    ,'Purpose:*'    :   'purpose'
    ,'from:'    :   'from'
    ,'Date of Birth:*'  :   'date_of_birth'
    ,'Training / Seminar schedule'  :   'training_seminar_schedule'
    ,'Chinese Name:'    :   'chinese_name'
    ,'Flight Requests:*'    :   'flight_requests'
    ,'E-ticket / Confirmation from Travel Coordinator'  :   'E_ticket_confirmation_from_travel_coordinator'
    ,'Description:*'    :   'description2'
    ,'Departure Time / Flight:' :   'departure_time_flight'
    ,'Date of Expiry:*' :   'date_of_expiry'
    ,'Arrival Time / Flight:'   :   'arrival_time_flight'
    ,'WBS number*'  :   'WBS_number'
    ,'Charge to other cost center *'    :   'charge_to_other_cost_center'
    ,'Remark'   :   'remark'
    ,'HoD Approval(for special requisition)'    :   'HoD_approval_for_special_requisition'
    ,'Replace Company'  :   'replace_company'
    ,'Travel Auditor*'  :   'travel_auditor'
    ,'Additional comment'   :   'additional_comment'
    ,'Days Before Go'   :   'days_before_go'
    ,'Company Code' :   'company_code'
    ,'Cost Center'  :   'cost_center'
    ,'CWT'  :   'CWT'
    ,'Pick-up Service:*'    :   'pick_up_service'
    ,'Rental Car for oversea:*' :   'rental_car_for_oversea'
    ,'Dates Between'    :   'dates_between'
}   

for old_name , new_name in cols.items():
    df = df.withColumnRenamed(old_name, new_name)

# todo 3: 数据处理      
formatted_endtime = datetime.now()
formatted_endtime += timedelta(hours=8)
etl_load_time = formatted_endtime.strftime("%Y-%m-%d %H:%M:%S")

df = df.select(list(cols.values())) \
        .dropDuplicates() \
        .withColumn('etl_load_time', F.lit(etl_load_time)) \
        .withColumn('personnel_number',F.regexp_replace(df['personnel_number'],'\'','')) \
        .toPandas()

display(df)

In [19]:
# todo 4：数据落盘
save_path = 'abfss://data-warehouse-dwd@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/dwd_fi_te_travel_app_header_cn.parquet'
df.to_parquet(save_path,index=False)

In [20]:
# todo 5：数据落盘csv
save_path = 'abfss://data-warehouse-dwd@dlsaaddpnorth3001.dfs.core.chinacloudapi.cn/dwd_fi_te_travel_app_header_cn.csv'
df.to_csv(save_path,index=False,header=True)